## SPARK SUBMIT

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Streaming').enableHiveSupport().getOrCreate()

In [3]:
spark

In [9]:
def loadData (qtde):
    list = []
    for x in range (qtde):
        print(x)
        r = requests.get('https://random-data-api.com/api/v2/users')
        list.append(r.json())
        req = spark.read.json(spark.sparkContext.parallelize(list))
        req = req.select( \
         'email' \
        ,'first_name' \
        ,'last_name' \
        ,'gender' \
        ,'id' \
        ,'username' \
                 )
    return req

In [10]:
df = loadData(10)

0
1
2
3
4
5
6
7
8
9


In [12]:
df.repartition(1).write.parquet('/datalake/raw/api',mode='append')

#### executar no terminal
 spark-submit app.py

## STREAMING

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Streaming').enableHiveSupport().getOrCreate()

In [2]:
df = spark \
    .readStream \
    .format('kafka') \
    .option('kafka.bootstrap.servers','kafka-broker:9092') \
    .option('subscribe','atividade') \
    .option('startingOffsets','earliest') \
    .option('kafka.group.id','spark3') \
    .load()

In [45]:
type(df)

pyspark.sql.dataframe.DataFrame

In [4]:
df.isStreaming

True

In [5]:
from pyspark.sql.functions import expr, from_json, col
from pyspark.sql.types import *

In [6]:
schema = StructType([
    StructField("activity", StringType()),
    StructField("type", StringType()),
    StructField("participants", IntegerType()),
    StructField("price", DoubleType()),
    StructField("link", StringType()),
    StructField("key", StringType()),
    StructField("accessibility", DoubleType())
                    ])

In [7]:
schema

StructType([StructField('activity', StringType(), True), StructField('type', StringType(), True), StructField('participants', IntegerType(), True), StructField('price', DoubleType(), True), StructField('link', StringType(), True), StructField('key', StringType(), True), StructField('accessibility', DoubleType(), True)])

In [24]:
value_df = df.select(from_json(col('value').cast('string'),schema).alias('value'))

In [25]:
type(value_df)

pyspark.sql.dataframe.DataFrame

In [26]:
exploded = value_df.selectExpr('value.activity','value.type','value.key','value.participants','value.price','value.link','value.accessibility')

### SALVAR NA MEMÓRIA

In [27]:
query = exploded.writeStream \
    .queryName('qraw') \
    .format('memory') \
    .start()

In [28]:
query.status

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [47]:
spark.sql('select * from qraw').count()

98

In [48]:
query.stop()

### SALVAR EM DISCO

In [51]:
saveDS = exploded.writeStream \
    .format('json') \
    .option('path','/datalake/raw/streaming') \
    .option('checkpointLocation','/spark/chkpoint') \
    .outputMode('append') \
    .queryName('SS WRITE') \
    .start()